<a href="https://colab.research.google.com/github/Dipak703/Clustering-with-sk/blob/main/chatgen_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.layers import LSTM, Bidirectional
from keras.optimizers import Adam

import json

In [ ]:
token = Tokenizer(oov_token="$_$")

In [ ]:
with open("dialogs.txt",'r') as da:
    data = da.read()
    text = data.lower().split( "\n")
texts =[]
ptext =[]
for items in text:
    ptext.append(items.split("\t"))
for item in ptext:
    texts.append(item[0])
    texts.append(item[1])
print(len(text))
print(texts)


3725
['hi, how are you doing?', "i'm fine. how about yourself?", "i'm fine. how about yourself?", "i'm pretty good. thanks for asking.", "i'm pretty good. thanks for asking.", 'no problem. so how have you been?', 'no problem. so how have you been?', "i've been great. what about you?", "i've been great. what about you?", "i've been good. i'm in school right now.", "i've been good. i'm in school right now.", 'what school do you go to?', 'what school do you go to?', 'i go to pcc.', 'i go to pcc.', 'do you like it there?', 'do you like it there?', "it's okay. it's a really big campus.", "it's okay. it's a really big campus.", 'good luck with school.', 'good luck with school.', 'thank you very much.', "how's it going?", "i'm doing well. how about you?", "i'm doing well. how about you?", 'never better, thanks.', 'never better, thanks.', 'so how have you been lately?', 'so how have you been lately?', "i've actually been pretty good. you?", "i've actually been pretty good. you?", "i'm actually

In [ ]:
# Serializing json
json_object = json.dumps(token.word_index)

# Writing to sample.json
with open("dailogtoken.json", "w") as outfile:
    outfile.write(json_object)


In [ ]:
token.fit_on_texts(texts)
print(token.word_index)


{'$_$': 1, 'i': 2, 'you': 3, 'the': 4, 'to': 5, 'a': 6, 'it': 7, 'that': 8, 'do': 9, 'what': 10, 'is': 11, 'of': 12, 'and': 13, 'have': 14, 'are': 15, 'in': 16, 'they': 17, 'was': 18, "it's": 19, 'did': 20, 'so': 21, 'like': 22, 'yes': 23, 'for': 24, 'my': 25, "don't": 26, 'but': 27, 'he': 28, 'no': 29, 'me': 30, 'be': 31, "i'm": 32, "that's": 33, 'about': 34, 'we': 35, 'on': 36, 'how': 37, 'go': 38, 'not': 39, 'think': 40, 'too': 41, 'why': 42, 'your': 43, 'good': 44, 'going': 45, 'will': 46, 'with': 47, 'want': 48, 'really': 49, 'get': 50, 'know': 51, 'well': 52, 'all': 53, 'at': 54, 'there': 55, 'one': 56, "i'll": 57, 'just': 58, 'can': 59, 'this': 60, 'would': 61, 'if': 62, "you're": 63, 'people': 64, 'see': 65, 'then': 66, 'she': 67, 'right': 68, 'nice': 69, "didn't": 70, 'out': 71, 'should': 72, "what's": 73, 'time': 74, 'oh': 75, 'need': 76, 'her': 77, 'money': 78, 'maybe': 79, 'him': 80, 'course': 81, 'lot': 82, 'up': 83, 'when': 84, 'or': 85, 'much': 86, 'okay': 87, 'day': 88,

In [ ]:
twords = len(token.word_index)+1

In [ ]:
input_seq=[]
listofseq=[]
for line in texts:
    token_seq = token.texts_to_sequences([line])[0]
    listofseq.append(token_seq)
    length=len(token_seq)
real_seq = list(filter(None, listofseq))
for seq in real_seq:
    size = len(seq)
    for i in range(1,size):
        input_seq.append(seq[:i+1])

In [ ]:
max_length = max([len(x) for x in real_seq])
print(len(input_seq))

40643


In [ ]:
input_seq = np.array(pad_sequences(input_seq,maxlen= max_length,padding ='pre'))
print(input_seq[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 1523   37]


In [ ]:
train = input_seq[:,:-1]
label = input_seq[:,-1]
y = tf.keras.utils.to_categorical(label,num_classes=twords)

In [ ]:
print(y.shape)

(40643, 2521)


In [ ]:
# bhai plz is ko barbar run mat ker new model bante hai
model1 = keras.Sequential([keras.layers.Embedding(twords,240,input_length = max_length-1),
                           keras.layers.Bidirectional(LSTM(150)),
                           keras.layers.Dense(twords, activation='softmax') ])
model1.summary()
adam =Adam(learning_rate =0.1)
model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 240)           605040    
                                                                 
 bidirectional (Bidirection  (None, 300)               469200    
 al)                                                             
                                                                 
 dense (Dense)               (None, 2521)              758821    
                                                                 
Total params: 1833061 (6.99 MB)
Trainable params: 1833061 (6.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# final_model = tf.keras.models.load_model('chatgen1.keras')

In [ ]:
history = model1.fit(train[:10000],y[:10000],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 16s 35ms/step - loss: 14.8579 - accuracy: 0.0299
Epoch 2/50
313/313 [==============================] - 3s 11ms/step - loss: 14.2544 - accuracy: 0.0565
Epoch 3/50
313/313 [==============================] - 3s 9ms/step - loss: 14.5283 - accuracy: 0.0782
Epoch 4/50
313/313 [==============================] - 3s 9ms/step - loss: 14.4173 - accuracy: 0.0860
Epoch 5/50
313/313 [==============================] - 3s 9ms/step - loss: 14.9498 - accuracy: 0.0941
Epoch 6/50
313/313 [==============================] - 3s 9ms/step - loss: 14.1658 - accuracy: 0.1058
Epoch 7/50
313/313 [==============================] - 3s 8ms/step - loss: 14.8883 - accuracy: 0.1182
Epoch 8/50
313/313 [==============================] - 3s 9ms/step - loss: 15.6973 - accuracy: 0.1190
Epoch 9/50
313/313 [==============================] - 4s 13ms/step - loss: 15.2714 - accuracy: 0.1333
Epoch 10/50
313/313 [==============================] - 3s 10ms/step - loss: 15.5277 - a

In [ ]:
model1.fit(train[10000:20000],y[10000:20000],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 3s 9ms/step - loss: 11.8890 - accuracy: 0.4356
Epoch 2/50
313/313 [==============================] - 3s 8ms/step - loss: 11.9986 - accuracy: 0.4332
Epoch 3/50
313/313 [==============================] - 3s 9ms/step - loss: 10.2695 - accuracy: 0.4684
Epoch 4/50
313/313 [==============================] - 3s 10ms/step - loss: 9.9588 - accuracy: 0.4740
Epoch 5/50
313/313 [==============================] - 2s 8ms/step - loss: 10.4971 - accuracy: 0.4739
Epoch 6/50
313/313 [==============================] - 2s 8ms/step - loss: 10.9685 - accuracy: 0.4566
Epoch 7/50
313/313 [==============================] - 2s 8ms/step - loss: 10.1876 - accuracy: 0.4826
Epoch 8/50
313/313 [==============================] - 3s 10ms/step - loss: 10.9552 - accuracy: 0.4676
Epoch 9/50
313/313 [==============================] - 3s 9ms/step - loss: 11.8850 - accuracy: 0.4409
Epoch 10/50
313/313 [==============================] - 3s 8ms/step - loss: 12.1321 - accur

In [ ]:
model1.fit(train[20000:30000],y[20000:30000],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 2s 8ms/step - loss: 90.7344 - accuracy: 0.0661
Epoch 2/50
313/313 [==============================] - 3s 10ms/step - loss: 45.1408 - accuracy: 0.1579
Epoch 3/50
313/313 [==============================] - 2s 8ms/step - loss: 31.0509 - accuracy: 0.2140
Epoch 4/50
313/313 [==============================] - 3s 8ms/step - loss: 23.1978 - accuracy: 0.2773
Epoch 5/50
313/313 [==============================] - 2s 7ms/step - loss: 19.0610 - accuracy: 0.3171
Epoch 6/50
313/313 [==============================] - 2s 8ms/step - loss: 15.9970 - accuracy: 0.3563
Epoch 7/50
313/313 [==============================] - 3s 10ms/step - loss: 13.9339 - accuracy: 0.3920
Epoch 8/50
313/313 [==============================] - 2s 8ms/step - loss: 13.0264 - accuracy: 0.4071
Epoch 9/50
313/313 [==============================] - 2s 7ms/step - loss: 13.3477 - accuracy: 0.3882
Epoch 10/50
313/313 [==============================] - 2s 7ms/step - loss: 12.7123 - accu

In [ ]:
model1.fit(train[30000:],y[30000:],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 3s 9ms/step - loss: 54.1274 - accuracy: 0.0812
Epoch 2/50
313/313 [==============================] - 2s 8ms/step - loss: 28.5514 - accuracy: 0.1499
Epoch 3/50
313/313 [==============================] - 3s 11ms/step - loss: 20.6425 - accuracy: 0.2137
Epoch 4/50
313/313 [==============================] - 2s 8ms/step - loss: 16.7302 - accuracy: 0.2641
Epoch 5/50
313/313 [==============================] - 2s 8ms/step - loss: 14.6050 - accuracy: 0.3000
Epoch 6/50
313/313 [==============================] - 2s 8ms/step - loss: 13.4315 - accuracy: 0.3342
Epoch 7/50
313/313 [==============================] - 2s 8ms/step - loss: 14.2792 - accuracy: 0.3137
Epoch 8/50
313/313 [==============================] - 3s 10ms/step - loss: 13.7934 - accuracy: 0.3253
Epoch 9/50
313/313 [==============================] - 2s 8ms/step - loss: 12.8080 - accuracy: 0.3449
Epoch 10/50
313/313 [==============================] - 2s 8ms/step - loss: 13.3735 - accu

In [ ]:
model1.save("dialogbot.keras")

In [ ]:
print(type(token.word_index))
print((token.word_index))
def get_key(val):
   for key, value in token.word_index.items():
        if val == value:
            return key

<class 'dict'>
{'$_$': 1, 'i': 2, 'you': 3, 'the': 4, 'to': 5, 'a': 6, 'it': 7, 'that': 8, 'do': 9, 'what': 10, 'is': 11, 'of': 12, 'and': 13, 'have': 14, 'are': 15, 'in': 16, 'they': 17, 'was': 18, "it's": 19, 'did': 20, 'so': 21, 'like': 22, 'yes': 23, 'for': 24, 'my': 25, "don't": 26, 'but': 27, 'he': 28, 'no': 29, 'me': 30, 'be': 31, "i'm": 32, "that's": 33, 'about': 34, 'we': 35, 'on': 36, 'how': 37, 'go': 38, 'not': 39, 'think': 40, 'too': 41, 'why': 42, 'your': 43, 'good': 44, 'going': 45, 'will': 46, 'with': 47, 'want': 48, 'really': 49, 'get': 50, 'know': 51, 'well': 52, 'all': 53, 'at': 54, 'there': 55, 'one': 56, "i'll": 57, 'just': 58, 'can': 59, 'this': 60, 'would': 61, 'if': 62, "you're": 63, 'people': 64, 'see': 65, 'then': 66, 'she': 67, 'right': 68, 'nice': 69, "didn't": 70, 'out': 71, 'should': 72, "what's": 73, 'time': 74, 'oh': 75, 'need': 76, 'her': 77, 'money': 78, 'maybe': 79, 'him': 80, 'course': 81, 'lot': 82, 'up': 83, 'when': 84, 'or': 85, 'much': 86, 'okay':

In [ ]:
seed_text = "i am "
next_words = 10
for _ in range(next_words):
  token_list = token.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')
  predicted = model1.predict(token_list, verbose=0)
  # print(max(predicted[0]))
  a = max(predicted[0])
  predicted=predicted.tolist()
  b = predicted[0].index(a)
  # print(b)
  output_word = get_key(b)
  seed_text += " "+ output_word

print(seed_text)
# with open("output2.txt",'w') as f:
#     f.write(seed_text)

i am  on the floor go back to to watch judge judy
